<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import psycopg2

import requests
import io
from bs4 import BeautifulSoup

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query = f'''
select count(*) from vacancies
'''
pd.read_sql_query(query, connection)

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [5]:
query = f'''
select count(*) from employers
'''
pd.read_sql_query(query, connection)

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [6]:
query = f'''
select count(*) from areas
'''
pd.read_sql_query(query, connection)

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [7]:
query = f'''
select count(*) from industries
'''
pd.read_sql_query(query, connection)

,count
0,294


***

Кол-во вакансий всего в 2 раза болше кол-ва работодателей. Похоже, что компании с большим кол-вом вакансий HH не пользуются, либо есть много работодателей без вакансий

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [8]:
query = f'''
select a.name area,
       count(*) cnt
  from vacancies v
  join areas a on a.id = v.area_id
 group by a.name
 order by cnt desc  
'''
pd.read_sql_query(query, connection)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [9]:
query = f'''
select count(*) cnt
  from vacancies v
 where v.salary_from is not null
    or v.salary_to is not null
'''
pd.read_sql_query(query, connection)

,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [10]:
query = f'''
select round(avg(v.salary_from)) avg_salary_from,
       round(avg(v.salary_to)) avg_salary_to
  from vacancies v
'''
pd.read_sql_query(query, connection)

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [11]:
query = f'''
select schedule,
       employment,
       count(*) cnt
  from vacancies v
 group by schedule,
          employment
 order by cnt desc
'''
pd.read_sql_query(query, connection)

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [12]:
query = f'''
select experience,
       count(*) cnt
  from vacancies v
 group by experience
 order by cnt
'''
pd.read_sql_query(query, connection)

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Большинство вакансий в крупных городах. Приверно у половины вакансий не заполнено не одно из полей зарплаты. Большинство вакансий на полный день, полную занятость. Больше 15% вакансий для удаленной работы. В основном ищут людей с небольшим опытом работы. Опытных сотрудников по объявлениям практически не ищут.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [13]:
query = f'''
select e.name,
       count(*) cnt
  from employers e
  join vacancies v on v.employer_id = e.id
 group by e.id
 order by cnt desc
'''
pd.read_sql_query(query, connection)

,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14901,Евсеев Сергей Владимирович,1
14902,Кадровые технологии,1
14903,ЛОМБАРД 24,1
14904,Авто-Компонент ТД,1


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [14]:
query = f'''
with employers as (
    select e.area,
           count(*) cnt
      from employers e
     group by e.area
), 
vacancies as (
    select v.area_id,
           count(*) cnt
      from vacancies v
     group by v.area_id
)
select a.name area,
       coalesce(e.cnt, 0) employers,
       coalesce(v.cnt, 0) vacancies
  from areas a
  left join employers e on e.area = a.id
  left join vacancies v on v.area_id = a.id
 order by vacancies, employers desc
'''
pd.read_sql_query(query, connection)

,area,employers,vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [15]:
# query = f'''
# select e.name,
#        count(*) cnt
#   from (
#     select distinct e.name, v.area_id
#       from employers e
#       join vacancies v on v.employer_id = e.id
#   ) e
#  group by e.name
#  order by cnt desc
# '''

query = f'''
with employer_vacancy_area as (
  select distinct e.name, v.area_id
    from employers e
    join vacancies v on v.employer_id = e.id
)
select e.name,
       count(*) cnt
  from employer_vacancy_area e
 group by e.name
 order by cnt desc
'''

pd.read_sql_query(query, connection)

,name,cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,Интеркросс,1
14762,Русская дымка,1
14763,Автономная некоммерческая организация Московск...,1
14764,Транспортный холдинг города Алматы,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [16]:
query = f'''
select count(*)
  from (
      select id from employers
      except
      select employer_id from employers_industries
  ) e
'''
pd.read_sql_query(query, connection)

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [17]:
query = f'''
select e.name
  from employers e
  join employers_industries i on i.employer_id = e.id
 group by e.id, e.name
having count(*) = 4
 order by e.name
offset 2
 limit 1'''
pd.read_sql_query(query, connection)

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [18]:
query = f'''
select count(*)
  from industries i
  join employers_industries ei on ei.industry_id = i.id
 where i.name = 'Разработка программного обеспечения'
'''
pd.read_sql_query(query, connection)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [19]:
response = requests.get('https://ru.wikipedia.org/wiki/Города-миллионеры_России')

# page = BeautifulSoup(response.text, 'html.parser')
# table = page.find(id='Список_городов').parent.parent.table
# rows = table.find_all('tr')[1:]
# m_cities = [tr.find_all('td')[1].text for tr in rows]

df = pd.read_html(io.StringIO(response.text))
m_cities = df[0]['Город'].to_list()

m_cities


['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Красноярск',
 'Нижний Новгород',
 'Челябинск',
 'Уфа',
 'Самара',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [20]:
query = f'''
with cities as (
    select a.name, count(*) cnt
      from vacancies v
      join areas a on a.id = v.area_id
      join employers e on e.id = v.employer_id
     where e.name = 'Яндекс'
       and a.name = any(%(m_cities)s)
     group by a.name
)  
select * from cities
union all
select 'Total', sum(cnt) from cities
order by 2
'''
pd.read_sql_query(query, connection, params={'m_cities': m_cities})

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

В лидерах работодатели из IT сферы. Работодатели предпочитают указывать конкретный город в вакансиях а не страну или регион - это странно, учитывая что ~15% вакансий для удаленной работы. Только два лидера по кол-ву вакансий также лидируют по кол-ву регионов. У 36% работодателей не указана сфера деятельности, только 15% работодателей работают в сфере IT. Только четверть вакансий Яндекс для городов миллионников.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [21]:
query = f'''
select count(*)
  from vacancies v
 where v.name ilike '%данн%'
    or v.name ilike '%data%'
'''
pd.read_sql_query(query, connection)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [22]:
query = f'''
select count(*)
  from vacancies v
 where (v.name ilike '%data scientist%'
        or v.name ilike '%data science%'
        or v.name ilike '%исследователь данных%'
        or (v.name ilike '%ml%' and v.name not ilike '%html%')
        or v.name ilike '%machine learning%'
        or v.name ilike '%машинн%обучен%')
   and (v.name ilike '%junior%'
        or v.experience = 'Нет опыта'
        or v.schedule = 'Стажировка')
'''
pd.read_sql_query(query, connection)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [23]:
query = f'''
select count(*)
  from vacancies v
 where (v.name ilike '%data scientist%'
        or v.name ilike '%data science%'
        or v.name ilike '%исследователь данных%'
        or (v.name ilike '%ml%' and v.name not ilike '%html%')
        or v.name ilike '%machine learning%'
        or v.name ilike '%машинн%обучен%')
   and (v.key_skills ilike '%sql%'
        or v.key_skills ilike '%postgres%')    
'''
pd.read_sql_query(query, connection)

,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [24]:
query = f'''
select count(*)
  from vacancies v
 where (v.name ilike '%data scientist%'
        or v.name ilike '%data science%'
        or v.name ilike '%исследователь данных%'
        or (v.name ilike '%ml%' and v.name not ilike '%html%')
        or v.name ilike '%machine learning%'
        or v.name ilike '%машинн%обучен%')
   and v.key_skills ilike '%python%'    
'''
pd.read_sql_query(query, connection)

,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [25]:
query = f'''
select round(avg(length(v.key_skills) - length(replace(v.key_skills, chr(9), '')) + 1), 2) key_skills_avg
  from vacancies v
 where (v.name ilike '%data scientist%'
        or v.name ilike '%data science%'
        or v.name ilike '%исследователь данных%'
        or (v.name like '%ML%' and v.name not ilike '%html%')
        or v.name ilike '%machine learning%'
        or v.name ilike '%машинн%обучен%')
'''
pd.read_sql_query(query, connection)

,key_skills_avg
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [26]:
query = f'''
 select experience, round(avg(salary)) salary_avg
  from (
     select v.experience, (coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from)) / 2 salary
      from vacancies v
     where (v.name ilike '%data scientist%'
            or v.name ilike '%data science%'
            or v.name ilike '%исследователь данных%'
            or (v.name like '%ML%' and v.name not ilike '%html%')
            or v.name ilike '%machine learning%'
            or v.name ilike '%машинн%обучен%')
  ) t
 group by experience
'''
pd.read_sql_query(query, connection)

,experience,salary_avg
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

~3.6% вакансий связаны с данными. Из вакансий связаных с данными ~3% доступны для начинающих. Из вакансий связаных с данными ~10% требуют знания SQL/Postgres, ~20% требуют Python - т.е. это востребованые навыки. В целом вакансии связаные с данными требуют разносторонней подготовки - ~6.5 навыков на вакансию. Человек с опытом от 3 до 6 лет может расчитывать на зарплату хорошего курьера или таксиста :)

# Общий вывод по проекту

In [70]:
query = f'''
with vacancy_industries as (
     select i.name industry, count(*) cnt
       from vacancies v
       join employers e on e.id = v.employer_id
       left join employers_industries ei on ei.employer_id = e.id
       left join industries i on i.id = ei.industry_id
      group by i.name
),
vacancies_total as (
     select count(*) total from vacancies
)
select industry, round(cnt * 100.0 /total, 2) mentioned_in_pct_of_vacancies
  from vacancy_industries,
       vacancies_total
 order by 2 desc
'''
display(pd.read_sql_query(query, connection).head(20))

,industry,mentioned_in_pct_of_vacancies
0,Разработка программного обеспечения,25.41
1,"Системная интеграция, автоматизации технологи...",22.43
2,None,20.50
3,"Интернет-компания (поисковики, платежные систе...",13.04
4,Банк,5.57
5,Мобильная связь,3.22
6,Кадровые агентства,2.32
7,Консалтинговые услуги,2.31
8,Интернет-провайдер,2.23
9,Розничная сеть (продуктовая),2.10


In [66]:
query = f'''
with it_vacancies as (
     select count(distinct v.id) cnt
       from vacancies v
       join employers e on e.id = v.employer_id
       left join employers_industries ei on ei.employer_id = e.id
       left join industries i on i.id = ei.industry_id
     where i.name in ('Разработка программного обеспечения',
                       'Системная интеграция,  автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг')
),
vacancies_total as (
     select count(*) total from vacancies
)
select cnt * 100 /total it_vacancies_pct
  from it_vacancies,
       vacancies_total
'''
pd.read_sql_query(query, connection)

,it_vacancies_pct
0,34


Более 34% вакансий связано с IT.
1. Либо HH нишевый сайт, либо это реальное распределение в целом в экономике и такой процент населения занят в этой сфере. Интересно было бы посмотреть какую-нибудь статистику отражающую выборку по всем профессиям/сферам деятельности.
2. 3.6% вакансий связано с данными. Подозреваю, что данные немного устарели, но даже по ним работа с данными составляет более 10% IT вакансий. Видимо, перспективная область. 
